In [2]:
import girderformindlogger
import girder_client as gc
import pandas as pd
import numpy as np

In [3]:
girder = gc.GirderClient(host="dev.mindlogger.org")

# Plan

- [x] create a new user (userA)
- [x] get userA's applets and make sure the list is empty
- [ ] userA creates an applet with an activity-set-url (url TBD)
- [ ] get userA's applets and make sure we have the applet they just created
- [ ] create userB
- [ ] userA invites userB to the applet
- [ ] make sure userB has an invite
- [ ] userB accepts their invite
- [ ] list userB's applets
- [ ] userA invites userC (by email) to the applet
- [ ] create userC with the same email
- [ ] check that userC has an invite
- [ ] accept userC's applet invite
- [ ] check userC's applets
- [ ] userA, B and C submit responses to each activity in the applet
- [ ] get responses for each user in the last 7 days
- [ ] user B removes the applet without removing data
- [ ] user C removes the applet AND removes data
- [x] delete userA, userB, and userC


# Create a new user

In [4]:
def testCreateUser():
    randomUser = np.random.randint(1000000)
    firstName = 'test'
    lastName = 'user'
    # NOTE: girder makes login and email lowercase!!!
    login = 'testuser{}'.format(randomUser)
    email = 'testemail{}@testemail.com'.format(randomUser)
    password = 'password'
    createUser = girder.post('user', parameters=dict(firstName=firstName,
                                        lastName=lastName,
                                        login=login,
                                        email=email,
                                        password=password
                                       )
               )
    # assert 'authToken' in createUser.keys(), 'user has no token, {}'.format(createUser)
    assert createUser['email'] == email, 'email does not match, {} {}'.format(createUser['email'], email)
    assert createUser['firstName'] == firstName, 'firstName does not match'
    assert createUser['lastName'] == lastName, 'lastName does not match'
    assert createUser['login'] == login, 'login does not match, {} {}'.format(createUser['login'], login)
    assert createUser['admin'] == False, 'user is an admin'
    assert createUser['_modelType'] == 'user', 'model is not user'
    assert createUser['public'] == False, 'user is public!'

    return createUser

In [5]:
new_user = testCreateUser()
girder.authenticate(username=new_user['login'], password='password')

{'_accessLevel': 2,
 '_id': '5d828bf7a2a2577370c218f4',
 '_modelType': 'user',
 'admin': False,
 'created': '2019-09-18T19:56:39.230000+00:00',
 'creatorId': '5d828bf7a2a2577370c218f4',
 'email': 'testemail573137@testemail.com',
 'emailVerified': False,
 'firstName': 'test',
 'lastName': 'user',
 'login': 'testuser573137',
 'otp': False,
 'public': False,
 'size': 0,
 'status': 'enabled'}

# Get applets when the user doesn't have any

In [6]:
def getAppletsNewUser(new_user):
    girder.authenticate(username=new_user['login'], password='password')
    appletList = girder.get('user/applets')
    assert len(appletList) == 0, 'a new user should have an empty list of applets. we get {}'.format(appletList)
    return 1

In [7]:
getAppletsNewUser(new_user)

1

# Add an applet

In [8]:
new_user = testCreateUser()
girder.authenticate(username=new_user['login'], password='password')

# TODO: create an activity-set that JUST for testing.
# make sure it has all the weird qualities that can break


# for now, lets do the mindlogger demo
activitySetUrl = 'http://repronim.org/schema-standardization/activity-sets/mindlogger-demo/mindlogger-demo_schema'

ar = girder.post('applet', parameters=dict(activity_set_url = activitySetUrl, name='testActivitySet'))



In [9]:
ar

{'_id': '5d828c02a2a2577370c218f6',
 'access': {'groups': [],
  'users': [{'flags': [], 'id': '5d828bffa2a2577370c218f5', 'level': 2}]},
 'baseParentId': '5c6aec8e9c7ea5529ee58db2',
 'baseParentType': 'collection',
 'created': '2019-09-18T19:56:50.161273+00:00',
 'creatorId': '5d828bffa2a2577370c218f5',
 'description': '',
 'lowerName': 'testactivityset (1)',
 'meta': {'activitySet': {'_id': 'activitySet/None', 'url': None},
  'applet': {}},
 'name': 'testActivitySet (1)',
 'parentCollection': 'collection',
 'parentId': '5c6aec8e9c7ea5529ee58db2',
 'public': True,
 'roles': {'editor': {'groups': [{'id': '5d828c02a2a2577370c218f8'}],
   'users': []},
  'manager': {'groups': [{'id': '5d828c02a2a2577370c218f9'}], 'users': []},
  'reviewer': {'groups': [{'id': '5d828c03a2a2577370c218fa', 'subject': None}],
   'users': []},
  'user': {'groups': [{'id': '5d828c02a2a2577370c218f7', 'subject': None}],
   'users': []}},
 'size': 0,
 'updated': '2019-09-18T19:56:50.299207+00:00'}

In [10]:
girder.get('applet/{}'.format(ar['_id']))

HttpError: HTTP error 500: GET https://dev.mindlogger.org:443/api/v1/applet/5d828c02a2a2577370c218f6
Response text: {"message": "An unexpected error occurred on the server.", "type": "internal", "uid": "2c37fd5d-1042-46a8-857e-85b3d87a97cd"}

# Delete a user

In [11]:
def testDeleteUser(new_user):

    girder.authenticate(username=new_user['login'], password='password')

    deleteUser = girder.delete('user/{}'.format(new_user['_id']))

    assert deleteUser['message'] == 'Deleted user {}.'.format(new_user['login']), "{} does not equal {}".format(deleteUser['message'], 'Deleted user {}'.format(new_user['login']))
    
    return 1

In [12]:
testDeleteUser(new_user)

1